In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# New Section

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Checking if i have GPU Enabled
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [ ]:
 # 2 very important preprocessing is Resizing and Rescaling
batch_size = 32

In [ ]:
training_datagenarator= ImageDataGenerator(rescale=1./255,horizontal_flip=True,
    vertical_flip=True,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2,validation_split=0.1)

test_datagen= ImageDataGenerator(rescale=1./255,horizontal_flip=True,
    vertical_flip=True,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2)

#training_datagenarator

In [ ]:
# dividing the data into training and validation
train=training_datagenarator.flow_from_directory('/content/gdrive/MyDrive/DisasterModel/train', target_size=(224, 224),color_mode='rgb', class_mode='categorical', batch_size=batch_size,subset='training')

validation=training_datagenarator.flow_from_directory('/content/gdrive/MyDrive/DisasterModel/train', target_size=(224, 224),color_mode='rgb', class_mode='categorical', batch_size=batch_size,subset='validation')

#test = test_datagen.flow_from_directory('/content/gdrive/MyDrive/DisasterModel/test', target_size=(224, 224),color_mode='rgb', class_mode='categorical', batch_size=batch_size,shuffle=False)

Found 1253 images belonging to 2 classes.
Found 139 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization

In [ ]:
# Initializing CNN
cnn=tf.keras.models.Sequential()

# adding first layer
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=[224,224,3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))
cnn.add(Dropout(0.25))
# adding second layer
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))
cnn.add(Dropout(0.25))

cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))
cnn.add(Dropout(0.25))

cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))
cnn.add(Dropout(0.25))

# adding third layer Extra layer added
cnn.add(tf.keras.layers.Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=1))
cnn.add(Dropout(0.25))

# Flattening
cnn.add(tf.keras.layers.Flatten())
#cnn.add(tf.keras.layers.Dropout(0.3))
cnn.add(Dropout(0.5))
# Fully connected layer
cnn.add(tf.keras.layers.Dense(units=512,activation='relu'))
cnn.add(BatchNormalization())
# Output layers
cnn.add(tf.keras.layers.Dense(units=2,activation='softmax'))

In [ ]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 112, 112, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 56, 56, 64)        0

In [ ]:
from keras.callbacks import EarlyStopping

checkpoint=tf.keras.callbacks.ModelCheckpoint('/content/gdrive/MyDrive/output/model.h5',
                                           monitor='val_loss',mode="min",save_best_only = True)
# simple early stopping
#es = EarlyStopping(monitor='val_loss', mode='max', verbose=1)

In [ ]:
callbacks=checkpoint

In [ ]:
cnn.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

cnn.fit(train,validation_data=validation,epochs=100,
                  steps_per_epoch=train.samples//batch_size,
                  validation_steps=validation.samples//batch_size,
              callbacks=callbacks
                 )

Epoch 1/100
39/39 [==============================] - 479s 12s/step - loss: 1.0982 - accuracy: 0.5176 - val_loss: 0.6312 - val_accuracy: 0.6641
Epoch 2/100
39/39 [==============================] - 37s 948ms/step - loss: 0.7114 - accuracy: 0.5733 - val_loss: 0.5837 - val_accuracy: 0.7031
Epoch 3/100
39/39 [==============================] - 37s 956ms/step - loss: 0.6644 - accuracy: 0.6470 - val_loss: 0.5442 - val_accuracy: 0.7266
Epoch 4/100
39/39 [==============================] - 34s 870ms/step - loss: 0.6461 - accuracy: 0.6568 - val_loss: 0.8789 - val_accuracy: 0.5938
Epoch 5/100
39/39 [==============================] - 33s 847ms/step - loss: 0.6520 - accuracy: 0.6323 - val_loss: 0.5606 - val_accuracy: 0.7188
Epoch 6/100
39/39 [==============================] - 33s 841ms/step - loss: 0.5986 - accuracy: 0.6921 - val_loss: 1.0181 - val_accuracy: 0.4844
Epoch 7/100
39/39 [==============================] - 33s 844ms/step - loss: 0.6173 - accuracy: 0.6749 - val_loss: 0.6561 - val_accuracy: 

In [ ]:
from tensorflow.keras.models import load_model
cnn=load_model('/content/gdrive/MyDrive/output/model.h5')

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import PIL

In [ ]:
def pred(a):
    test_image=image.load_img(a,target_size=(224,224))
    test_image=image.img_to_array(test_image)
    test_image=test_image/255
    test_image=np.expand_dims(test_image,axis=0)
    result=cnn.predict(test_image)

    Catagories=['disaster','non-disaster']

    image_show=PIL.Image.open(a)
    plt.imshow(image_show)

    plt.title(Catagories[int(np.argmax(result))])
    plt.show()

In [ ]:
import os
base_path = "/content/gdrive/MyDrive/test"
a = os.listdir("/content/gdrive/MyDrive/test")
for i in a:
  path = base_path + '/' + i
  pred(path)